In [93]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
from webdriver_manager.chrome import ChromeDriverManager

In [94]:
# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\natha\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


# Scrape NASA Mars News

In [95]:
# Visit https://redplanetscience.com/
article_url = "https://redplanetscience.com/"
browser.visit(article_url)

time.sleep(1)

In [96]:
# Scape page into Soup
art_html = browser.html
art_soup = bs(art_html, "html.parser")

slide_elem = art_soup.select_one('div.list_text')

In [97]:
slide_elem

<div class="list_text">
<div class="list_date">August 1, 2021</div>
<div class="content_title">How NASA's Perseverance Mars Team Adjusted to Work in the Time of Coronavirus </div>
<div class="article_teaser_body">Like much of the rest of the world, the Mars rover team is pushing forward with its mission-critical work while putting the health and safety of their colleagues and community first.</div>
</div>

In [98]:
# Get latest news title
latest_news_title = slide_elem.find('div', class_='content_title').get_text()
latest_news_title

"How NASA's Perseverance Mars Team Adjusted to Work in the Time of Coronavirus "

In [99]:
# Get the paragraph text for latest new title
news_para = slide_elem.find('div', class_='article_teaser_body').get_text()
news_para

'Like much of the rest of the world, the Mars rover team is pushing forward with its mission-critical work while putting the health and safety of their colleagues and community first.'

# Scape Mars Space Images

In [122]:
# Visit https://spaceimages-mars.com/
image_url = "https://spaceimages-mars.com/"
browser.visit(image_url)

time.sleep(1)

In [123]:
# Scape image_page into Soup
image_html = browser.html
image_soup = bs(image_html, "html.parser")

In [155]:
# Find FULL IMAGE button
button_elem = image_soup.select_one('div.floating_text_area')
button_elem

<div class="floating_text_area">
<h2 class="brand_title">FEATURED IMAGE</h2>
<h1 class="media_feature_title">Dusty Space Cloud</h1>
<br/>
<a class="showimg fancybox-thumbs" href="image/featured/mars2.jpg" target="_blank"> <button class="btn btn-outline-light"> FULL IMAGE</button></a>
</div>

In [153]:
# Get FULL IMAGE link
full_image_button = button_elem.select_one('a').get("href")
full_image_button

'image/featured/mars2.jpg'

In [154]:
# Add FULL IMAGE link to base url to get featured image
featured_image_url = image_url + full_image_button
featured_image_url

'https://spaceimages-mars.com/image/featured/mars2.jpg'